# Angular Component Generator

This notebook leverages an embeddings model to analyze the code structure of this application, and generates a new Angular component with a foundational model that adheres to the proprietary tooling and best practices observed in the existing codebase. 

By using LangChain to manage prompt templates and employing Retrieval-Augmented Generation (RAG) to provide context around embeddings, we ensure that the generated code is both relevant and aligned with the existing codebase. This process demonstrates the capability of fine-tuning a model to follow specific development conventions and incorporate them into the generated code.

To run this notebook, ensure you have access to the following models within Bedrock:
- **anthropic.claude-3-sonnet-20240229-v1:0**
- **amazon.titan-embed-text-v1**

## Step 1: Environment Setup

Initialize all necessary libraries and create an instance of the AWS BedRock boto3 client. This step ensures that the environment is properly configured to interact with the Bedrock service.

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))

print("Connected to AWS BedRock")

## Step 2: Load and Analyze Angular Components

Load the Angular components from subdirectories within the `app` folder and use the embeddings model to analyze their structure. This step involves reading the files, creating embeddings, and storing them in a vector store for later retrieval.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os

def load_all_files_recursive(directory):
    documents = []
    subfolders = set()
    
    for root, dirs, files in os.walk(directory):
        dirs[:] = [d for d in dirs if not d.startswith('.')] # Skip hidden subfolders

        for file in files:
            filepath = os.path.join(root, file)

            try:
                loader = TextLoader(filepath, encoding="utf-8")
                loaded_docs = loader.load()

                # Filter out empty documents
                non_empty_docs = [doc for doc in loaded_docs if doc.page_content.strip()]
                documents.extend(non_empty_docs)

                print(f"Parsed file {filepath}")

            except UnicodeDecodeError:
                print(f"Skipping binary file or file with unknown encoding: {filepath}")

            except Exception as e:
                print(f"Error loading file {filepath}: {e}")

        # Add subfolder names to the set.
        # This will help train the model on all components, assume they are contained
        # within subfolders inside /src/app
        for subfolder in dirs:
            subfolders.add(os.path.relpath(os.path.join(root, subfolder), directory))

    return documents, ", ".join(sorted(subfolders))

# Load ALL files recursively
documents, subfolders = load_all_files_recursive("../src/app")
print(f"Loaded {len(documents)} documents.")

# Create embeddings
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Create vector store
vectorstore_faiss = FAISS.from_documents(documents, embedding=br_embeddings)
print("Vector store created.")

# Step 3: Create Component Knowledge Base
Add guidance around how to create a component to the vector DB.

In [ ]:
from langchain.document_loaders import TextLoader

# Load component knowledge from file
knowledge_file_path = "component_knowledge.md"
loader = TextLoader(knowledge_file_path, encoding="utf-8")
component_knowledge_doc = loader.load()

vectorstore_faiss.add_documents(component_knowledge_doc, metadata=metadata)
print("Component knowledge loaded.")

# Step 4: Create an Agent Graph
Create tools to query the vector DB above, as well as query JIRA for a given ticket. Attach the tools to a `StateGraph` and instruct the model to utilize the tools when generating a component.

In [ ]:
import requests
from typing import Literal
from IPython.display import Image, display
from langchain_aws import ChatBedrock
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

# Initialize chat model
chat_model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=bedrock_client,
    model_kwargs={
        "temperature": 0.2,
        "top_p": 0.1,
    },
)

# Create a JIRA lookup tool
@tool
def jira_lookup(ticket_id: str) -> str:
    "Useful for when you need to lookup information about a JIRA "
    "ticket. You should only call this function with a JIRA "
    "ticket ID, e.g. 'ABC-123'. "
    "The input to this tool should be a string that is a valid "
    "JIRA ticket ID"


    # This is a makeshift JIRA mapping to a github URL.
    # For a real implementation, this will be replaced with just your
    # JIRA base URL.
    jira_mapping = {
        "ABC-123": (
            "https://gist.githubusercontent.com/sayakb/"
            "338017679afd3b9f5d373e4f8c0a69df/raw/"
            "7c498945f7dbf821782ea5faf77e5ae7fdd23485/ABC-123.json"
        ),
        "DEF-456": (
            "https://gist.githubusercontent.com/sayakb/"
            "26c4b5829b341d51b1899bb12b1fbe92/raw/"
            "5422b089f2b4ebe0ffddf1f3016a82b08ee24985/DEF-456.json"
        )
    }

    if ticket_id not in jira_mapping:
        return "ERRNOTFOUND: Invalid JIRA ticket"

    url = jira_mapping.get(ticket_id)
    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    else:
        return f"ERRHTTP: Failed to retrieve content, status code {response.status_code}"

# Create a component structure tool
@tool
def component_struct() -> str:
    "Useful when creating an Angular component to determine "
    "the various filed that must be created"
    response_template = (
        "Component File Structure and other Non-Functional Requirements:"
        "\n\n{}\n\n"
        "Examples to use for the component content:"
        "\n\n{}\n\n"
    )

    hr = "\n\n============================================================\n\n"
    comp_struct = ""
    examples = ""

    for document in vectorstore_faiss.similarity_search("*", k=100):
        doc_source = document.metadata["source"]
        page_content = document.page_content

        if doc_source != "component_knowledge.md":
            examples += (
                f"Filename: {doc_source}\n"
                f"Contents:\n```\n{page_content}\n```"
                f"{hr}"
            )
        else:
            comp_struct = document.page_content

    return response_template.format(comp_struct, examples)

tools = [jira_lookup, component_struct]
tool_node = ToolNode(tools)
model_with_tools = chat_model.bind_tools(tools)

# Create a conditional edge in the agent graph
def next_step(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"

    return "__end__"

# Get information about the available tools
def ask_model_to_reason(state: MessagesState):
    messages = state["messages"]

    try:
        response = model_with_tools.invoke(messages)
    except Exception as e:
        return {"messages": [messages.append("Unable to invoke the model")]}

    return {"messages": [response]}

# Construct the agent graph
agent_graph = StateGraph(MessagesState)
agent_graph.add_node("agent", ask_model_to_reason)
agent_graph.add_node("tools", tool_node)
agent_graph.add_edge("__start__", "agent")
agent_graph.add_edge("tools", "agent")
agent_graph.add_conditional_edges("agent", next_step)
react_agent = agent_graph.compile()

display(Image(react_agent.get_graph().draw_mermaid_png()))

## Step 5: Create a ChatUX to Generate Code

This step defines the `ChatUX` class, which uses IPWidgets to create a user-friendly interface for inputting the component name and displaying the generated code.

In [ ]:
import ipywidgets as ipw
from IPython.display import display

class ChatUX:
    """ A chat UX for generating components """
    def __init__(self, react_agent):
        self.react_agent = react_agent
        self.component_txt = None
        self.jira_txt = None
        self.ipw_output = ipw.Output()

        self.prompt_template = (
            "Create a component named {} that adheres to the "
            "requirements listed in the JIRA ticket {}"
        )

        self.system_message = (
            "Generate Angular components as instructed by the user. "
            "Think step by step. Use the tools provided to determine "
            "the component file structure and to obtain ticket details "
            "from JIRA. If unable to find a JIRA ticket (ERRNOTFOUND) "
            "do not generate the code and ask the user to specify a "
            "valid JIRA ticket ID"
        )

    def display_box(self):
        with self.ipw_output:
            self.component_txt = ipw.Text(placeholder='Enter component name')
            self.jira_txt = ipw.Text(placeholder='Enter JIRA ticket ID')
            self.create_btn = ipw.Button(description="Create")
            self.create_btn.on_click(self.create_handler)

            display(ipw.Box(children = (
                self.component_txt,
                self.jira_txt,
                self.create_btn
            )))

    def start_chat(self):
        print("Welcome to the component generator!")
        self.display_box()
        display(self.ipw_output)

    def print_stream(self, stream):
        for item in stream:
            message = item["messages"][-1]

            if isinstance(message, tuple):
                print(message)
            else:
                message.pretty_print()

    def create_handler(self, _):
        with self.ipw_output:
            component_name = self.component_txt.value
            jira_ticket_id = self.jira_txt.value
            
            if len(component_name) > 0 and len(jira_ticket_id) > 0:
                print(f"Generating component with name {component_name} "
                      f"based on requirements from JIRA ticket {jira_ticket_id}...\n\n")

                formatted_query = self.prompt_template.format(component_name, jira_ticket_id)
                inputs = {"messages": [("system", self.system_message), ("user", formatted_query)]}
                config = {"recursion_limit": 15}
        
                self.print_stream(self.react_agent.stream(inputs, config, stream_mode="values"))
                self.component_txt.disabled = True
                self.jira_txt.disabled = True
                self.create_btn.disabled = True
                self.component_txt = None
                self.jira_txt = None
                self.display_box()
            else:
                print("Please enter the component name and JIRA ticket ID")

ux = ChatUX(react_agent=react_agent)
ux.start_chat()

## Conclusion

This successfully demonstrates the ability to tune a model to generate code in a specific format that aligns with a project's best practices and uses provided tools. This also demonstrates the ability to connect to external sites like JIRA to retrieve additional business intent.